In [157]:
import os
import tweepy as tw #be sure to "pip install tweepy"
import numpy as np
import pandas as pd
import time
from IPython import display
import pickle
import datetime

In [95]:
#import necessary keys from Twitter Developer API
CONSUMER_KEY = 'uUxsn1nVeotKTGiMXFe9ZV058'
CONSUMER_SECRET = 'EsKP4gYXWEeLjKc4CUBw5gwkzJnXut0kWeqM6RS0GqQ23uQzDX'
ACCESS_TOKEN = '1222371563594506240-usNlkp4rMMjc6E5r2Rab0A6eovPkPi'
ACCESS_TOKEN_SECRET = 'N6xLlu5KDcGMhNsJxYw47JRNAA7Q7B6XcYjyNTrRnaAka'

In [139]:
auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [140]:
def rate_limit_sleep(cursor):
    while True:
        try:
            yield cursor.next()
        except tw.RateLimitError:
            for i in range(15*60,0,-1):
                print("Rate Limit Exceeded\nReturning from sleep in {} minute(s) and {} seconds"\
                      .format(i//60,i%60),end='\r')
            time.sleep(1)
        except StopIteration:
            return

In [141]:
def metadata_df(search_api,hashtags,since,until, lat_long_only=True):
    #keep track of how many tweets with valid latitude and longitude are preserved
    total_tweets = 0
    latlong_tweets = 0
    
    #open current pickle if it exists to keep appending to it
    if os.path.exists("tweets.pkl"):
        df = pickle.load(open('tweets.pkl', 'rb'))
    #otherwise, initialize an empty dataframe with the desired columns
    else:
        df = pd.DataFrame(columns=['tweet_id',
                                   'created_at',
                                   'user_id',
                                   'user_name',
                                   'location',
                                   'country',
                                   'latitude',
                                   'longitude',
                                   'text'])
    
    # loop through tweet objects
    for tweet in tw.Cursor(api.search,
                                          q=hashtags,
                                          #lang="en",
                                          since=since,
                                          until=until,
                                          tweet_mode='extended',  
                                          count = 2000).items():
        
        #this just prints the tweets so we know the progress of pulling
        total_tweets += 1
        print("Total tweets pulled: ",total_tweets)
        print("Tweets with valid latitude/longitude: ",latlong_tweets)
        display.clear_output(wait=True)
        
        #NOTE: this line ensures that we only return tweets with valid locations/latitudes + longitudes
        if lat_long_only:
            if not tweet.place: continue
            
        #first, get location (city) and country depending on what info is available
        if tweet.place != None:
            location = tweet.place.full_name
            country = tweet.place.country_code
            
        else:
            location = tweet.user.location #if actual city doesn't exist, get location provided by the user
            country = np.nan
            
        #get latitude and longitude depending on whether an exact location or bounding box is provided
        if tweet.coordinates != None: #exact location
            longitude, latitude = tweet.coordinates['coordinates']
            
        elif tweet.place != None: #bounding box --> take center point (average) of box
            bounding_box = np.array(tweet.place.bounding_box.coordinates[0])
            longitude, latitude = np.mean(bounding_box, axis=0)
            
        else: #no location provided --> make nan
            longitude = np.nan
            latitude = np.nan
        
        latlong_tweets += 1
        print("Total tweets pulled: ",total_tweets)
        print("Tweets with valid latitude/longitude: ",latlong_tweets)
        display.clear_output(wait=True)

        # create and/or append this tweet to the dataframe
        df = df.append({'tweet_id':tweet.id,
                       'created_at':tweet.created_at,
                       'user_id':tweet.user.id,
                       'user_name':tweet.user.screen_name,
                       'location':location,
                       'country':country,
                       'latitude':latitude,
                       'longitude':longitude,
                       'text':tweet.full_text,
                      }, ignore_index=True)
        
        #pickle the intermediate result
        df.to_pickle('tweets.pkl')
        
    print("Total tweets pulled: ",total_tweets)
    print("Tweets with valid latitude/longitude: ",latlong_tweets) 
    
    return df

In [142]:
#if we hit the rate limit, pickle and save the current result, then after the rate limit refreshes,
#keep adding to the dataframe and append to the previous pickle each time

#collect tweets
hashtags = '#coronavirus OR #covid19 -filter:retweets'
#gets all tweets on 2020-03-24
df = metadata_df(api.search, hashtags, since='2020-03-24',until='2020-03-25',lat_long_only=True)
df

Rate limit reached. Sleeping for: 759


KeyboardInterrupt: 

In [128]:
#open pickle file to make sure it worked
pickle.load(open('tweets.pkl', 'rb'))

,tweet_id,created_at,user_id,user_name,location,country,latitude,longitude,text
0,1242387245983825920,2020-03-24 09:46:16,835984980,sub3chekal,"Lynnfield, MA",US,42.538193,-71.030541,At least one solo 9 mile run a day to keep men...
1,1242387243731427329,2020-03-24 09:46:16,2352462153,carlanna35,"Rovato, Lombardia",IT,45.548077,10.012376,"Lo stanno cercando,è subdolo,si nasconde nella..."
2,1242387242565488640,2020-03-24 09:46:15,2409628688,_MFCricket_,"Wells, England",GB,51.209998,-2.639966,Who gets your vote 🗳 - Boys Dev XI 🏏- 4 of the...
3,1242387237238603777,2020-03-24 09:46:14,417805251,ehjongsma,"Leucate, France",FR,42.907432,3.025044,Zijn de eerste positieve signalen en trendbreu...
4,1242387231287005184,2020-03-24 09:46:13,155202983,DrPascalMeier,"Chur, Switzerland",CH,46.853132,9.532746,"Still so many #Covidiots: downplaying, ignori..."
...,...,...,...,...,...,...,...,...,...
694,1242378742321815552,2020-03-24 09:12:29,2512684916,SilvanSilvaldo,Namibia,NA,-22.966418,18.432152,People are dying in Italy 😱😱\n#COVID19 mbela l...
695,1242378719190298625,2020-03-24 09:12:23,556203067,ZACHYTEDDY,Golf de yaoundé,CM,3.897443,11.498299,"not only did u impact the world music scene, b..."
696,1242378701586567168,2020-03-24 09:12:19,2824513470,KhannaPal,"Yamunanagar, India",IN,30.251444,77.339529,गोबर/मूत्र/ताली/थाली और 🔔 की आपार सफलता के बाद...
697,1242378687925764096,2020-03-24 09:12:16,930931380703305729,BitcoinMktJrnl,"Boston, MA",US,42.313670,-71.088712,The giant Democratic version of the #Coronavir...


In [130]:
#df.sort_values('created_at').reset_index() #this sorts the dataframe to increasing from earliest date

,index,tweet_id,created_at,user_id,user_name,location,country,latitude,longitude,text
0,698,1242378633206874112,2020-03-24 09:12:03,151568306,JohnsonManabat,"Quezon City, National Capital Region",PH,14.683012,121.062735,90 bagong kaso ng #COVID19 sa bansa naitala ng...
1,697,1242378687925764096,2020-03-24 09:12:16,930931380703305729,BitcoinMktJrnl,"Boston, MA",US,42.313670,-71.088712,The giant Democratic version of the #Coronavir...
2,696,1242378701586567168,2020-03-24 09:12:19,2824513470,KhannaPal,"Yamunanagar, India",IN,30.251444,77.339529,गोबर/मूत्र/ताली/थाली और 🔔 की आपार सफलता के बाद...
3,695,1242378719190298625,2020-03-24 09:12:23,556203067,ZACHYTEDDY,Golf de yaoundé,CM,3.897443,11.498299,"not only did u impact the world music scene, b..."
4,694,1242378742321815552,2020-03-24 09:12:29,2512684916,SilvanSilvaldo,Namibia,NA,-22.966418,18.432152,People are dying in Italy 😱😱\n#COVID19 mbela l...
...,...,...,...,...,...,...,...,...,...,...
694,4,1242387231287005184,2020-03-24 09:46:13,155202983,DrPascalMeier,"Chur, Switzerland",CH,46.853132,9.532746,"Still so many #Covidiots: downplaying, ignori..."
695,3,1242387237238603777,2020-03-24 09:46:14,417805251,ehjongsma,"Leucate, France",FR,42.907432,3.025044,Zijn de eerste positieve signalen en trendbreu...
696,2,1242387242565488640,2020-03-24 09:46:15,2409628688,_MFCricket_,"Wells, England",GB,51.209998,-2.639966,Who gets your vote 🗳 - Boys Dev XI 🏏- 4 of the...
697,1,1242387243731427329,2020-03-24 09:46:16,2352462153,carlanna35,"Rovato, Lombardia",IT,45.548077,10.012376,"Lo stanno cercando,è subdolo,si nasconde nella..."


In [168]:
(datetime.date.today()-datetime.timedelta(1)).strftime("%Y-%m-%d")

'2020-03-23'

In [155]:
#to get all the tweets in one day, since=today, until=tomorrow (even if in the future)
#TODAY IS MARCH 24 ==> 7 "days" before is since='2020-03-16', until '2020-03-24'
for tweet in tw.Cursor(api.search,
                      q=hashtags,
                      #lang="en",
                      since='2020-03-16', 
                      until='2020-03-24',
                      tweet_mode='extended').items(30):
    print(tweet.created_at)

2020-03-23 23:59:59
2020-03-23 23:59:59
2020-03-23 23:59:59
2020-03-23 23:59:59
2020-03-23 23:59:59
2020-03-23 23:59:59
2020-03-23 23:59:58
2020-03-23 23:59:58
2020-03-23 23:59:58
2020-03-23 23:59:57
2020-03-23 23:59:57
2020-03-23 23:59:57
2020-03-23 23:59:57
2020-03-23 23:59:57
2020-03-23 23:59:56
2020-03-23 23:59:56
2020-03-23 23:59:56
2020-03-23 23:59:56
2020-03-23 23:59:56
2020-03-23 23:59:56
2020-03-23 23:59:56
2020-03-23 23:59:56
2020-03-23 23:59:56
2020-03-23 23:59:55
2020-03-23 23:59:55
2020-03-23 23:59:55
2020-03-23 23:59:55
2020-03-23 23:59:55
2020-03-23 23:59:55
2020-03-23 23:59:55
